## ONNX Runtime 분석
- Scikit-learn to ONNX 모델 변환 (ONNX-ML)
- logreg_iris.onnx ORT 속성들 분석

![OnnxRuntime구조](/img/OnnxRuntime구조.png)

In [5]:
# Iris Data Load 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(112, 4) (38, 4) (112,) (38,)


In [7]:
# onnx model load
import onnx

# model load
onx_ml = onnx.load('logreg_iris.onnx')
print(type(onx_ml))

# protobuf 출력
onx_ml

<class 'onnx.onnx_ml_pb2.ModelProto'>


ir_version: 4
producer_name: "skl2onnx"
producer_version: "1.6.1"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "float_input"
    output: "label"
    output: "probability_tensor"
    name: "LinearClassifier"
    op_type: "LinearClassifier"
    attribute {
      name: "classlabels_ints"
      ints: 0
      ints: 1
      ints: 2
      type: INTS
    }
    attribute {
      name: "coefficients"
      floats: -0.3999278247356415
      floats: 0.8521245121955872
      floats: -2.3182997703552246
      floats: -0.9993020296096802
      floats: 0.46596747636795044
      floats: -0.333920419216156
      floats: -0.16528157889842987
      floats: -0.8876999020576477
      floats: -0.06603965908288956
      floats: -0.5182040929794312
      floats: 2.483581304550171
      floats: 1.8870019912719727
      type: FLOATS
    }
    attribute {
      name: "intercepts"
      floats: 9.224774360656738
      floats: 2.173171043395996
      floats: -11.397945404052734
    

In [8]:
# ONNX Runtime 결과
import onnxruntime as rt
import numpy as np

# 파일 읽어오기
sess = rt.InferenceSession(path_or_bytes = "logreg_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# predict
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

In [9]:
# Accuracy 평가지표
from sklearn.metrics import accuracy_score

# Predict에 대한 Accuracy Score
print('accuracy_score', float(accuracy_score(y_test, pred_onx))) 

accuracy_score 1.0


In [60]:
# input data에 대한 정보, output data에 대한 정보
print(sess.get_inputs()[0].name)
print(sess.get_inputs()[0].shape)
print(sess.get_inputs()[0].type)
print('-----------------')
print(sess.get_outputs()[0].name)
print(sess.get_outputs()[0].shape)
print(sess.get_outputs()[0].type)

float_input
[None, 4]
tensor(float)
-----------------
output_label
[1]
tensor(int64)


In [14]:
# Runtime Version, author
rt.__version__
rt.__author__

'1.0.0'

'Microsoft'

In [12]:
# Runtinme Function
rt.get_all_providers() # cpu, cuda, openvino, mkl-dnn(memory?)
rt.get_available_providers() 
rt.get_device() # cpu, gpu
rt.onnxruntime_validation.check_distro_info()

['TensorrtExecutionProvider',
 'CUDAExecutionProvider',
 'MKLDNNExecutionProvider',
 'NGRAPHExecutionProvider',
 'OpenVINOExecutionProvider',
 'NupharExecutionProvider',
 'BrainSliceExecutionProvider',
 'CPUExecutionProvider']

['CPUExecutionProvider']

'CPU'